In [ ]:
%%time 

###########################################################
# Parte 1
# Configurações gerais de log e warning
#############################################################

#Importação dos módulos desta parte
from datetime import datetime
import logging
import warnings

t0 = datetime.now()
#Criação e configuração do log
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG) 
logFormatter = logging.Formatter('"%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s"')
fh = logging.FileHandler('/home/luis/Downloads/Ex2-3 V{0}.log'.format(datetime.now()))
fh.setLevel(logging.DEBUG)
fh.setFormatter(logFormatter)
log.addHandler(fh)
ch = logging.StreamHandler()
ch.setFormatter(logFormatter)
ch.setLevel(logging.DEBUG)
log.addHandler(ch)

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
warnings.warn = warn
 
#log das versões utilizadas
import sklearn
import scipy

log.info("Versões utilizadas:")
log.info("Sklearn: %s" % sklearn.__version__)
log.info("Scipy: %s" % scipy.__version__)
    
    
log.info("Start!")

###########################################################
# Parte 2
# Importação da base QM7 e separação dos dados de treino e teste
#############################################################

#Importação dos módulos desta parte
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np

t1 = datetime.now()
log.info("")
log.info("------------------------------------------------------")
log.info("")
log.info("Início da parte 2: ")
log.info("Importação da base QM7 e separação dos dados de treino e teste")

#https://www.kaggle.com/mjmurphy28/predicting-atomization-energy-qm7
#The data file (.mat format, we recommend using scipy.io.loadmat for python users) contains five arrays:
#
#"X" - (7165 x 23 x 23), Coulomb matrices
#"T" - (7165), atomization energies (unit: kcal/mol)
#"P" - (5 x 1433), cross-validation splits as used in [Montavon et al. NIPS, 2012]
#"Z" - (7165 x 23), atomic charges
#"R" - (7165 x 23 x 3), cartesian coordinate (unit: Bohr) of each atom in the molecules
mat = loadmat('/home/luis/Dropbox/Pós Graduacao/qm7.mat')
X3D = mat['X'] # Coulomb matrices 3D (7165 x 23 x 23)
y = mat['T'].T #atomization energies


# Separação das partes de X
X_CM = X3D.reshape(7165,529)
#num_atoms = 23
#X_CM = np.zeros((X3D.shape[0], num_atoms*(num_atoms+1)//2), dtype=float)
#iu = np.triu_indices(num_atoms,k=0) 
#iu_dist = np.triu_indices(num_atoms,k=1) # for the pairwise distance matrix, all diagonol entries will be 0 
#X_eigs = np.zeros((X3D.shape[0], num_atoms), dtype=float)
#X_centralities = np.zeros((X3D.shape[0], num_atoms), dtype=float)
#X_interatomic_dist = np.zeros((X3D.shape[0], ((num_atoms*num_atoms)-num_atoms)//2), dtype=float) 
log.info("Feita a leitura do arquivo qm7.mat")

#Separação dos dados de treino e teste
train_X, val_X, train_y, val_y = train_test_split(X_CM, y, random_state=1)
log.info("Feita a separação dos dados de treino e teste")

#maca fim do bloco e tempo gasto
log.info("Tempo gasto na Parte 2: %s" % (datetime.now() - t1))

###########################################################
# Parte 3
# Avaliação dos parametros com o GridSearchCV
#############################################################


#Importação dos módulos desta parte
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd

t1 = datetime.now()
log.info("")
log.info("------------------------------------------------------")
log.info("")
log.info("Início da parte 3: ")
log.info("Avaliação dos parametros com o GridSearchCV")

# Array para guardar os melhores parâmetros de cada execução
best_params = []

# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 5
# gamma: 0.1, 0.3, 0.5, 0.8
# degree: Fibonacci de 1 a 5
# coef0: 0.1, 0.3, 0.5, 0.8
param_KR = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8]},  
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8],
                     'degree': [1, 2, 3],
                     'coef0': [0.1, 0.3, 0.5, 0.8]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8]}
                    ]
param_SVR = [{'kernel': ['linear'],  #o kernel linear só com um parametro demrou 20mn. para executar 
                     'C': [1]}, #, 2, 3, 5]}, 
                    {'kernel': ['rbf'], 
                     'C': [1], # 2, 3, 5], 
                     'gamma': [0.1]}, # 0.3, 0.5, 0.8]},  
                    # {'kernel': ['polynomial'], 
                    # 'C': [1], #2, 3, 5], 
                    # 'gamma': [0.1], # 0.3, 0.5, 0.8],
                    # 'degree': [1], #2, 3, 5],
                    # 'coef0': [0.1]}, # 0.3, 0.5, 0.8]},
                   # {'kernel': ['cosine'], 
                   #  'C': [1, 2, 3, 5]}, 
                   # {'kernel': ['laplacian'], 
                   #  'C': [1, 2, 3, 5], 
                   #  'gamma': [0.1, 0.3, 0.5, 0.8]}
                    ]
estimators = [{"name": "KernelRidge",
               "estimator": KernelRidge(),
              "parameters": param_KR},
              {"name": "SVR",
              "estimator": SVR(),
             "parameters": param_SVR},
          ]
log.info("Criados os parâmetros para os estimators: {0}".format(estimators))

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV: {0}".format(scores))

#laço para avaliação dos estimators
for estimator in estimators:
    # esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
    #mas foi usada em um laço de "for" para melhor clareza no log de execução
    for score in scores: 

        log.info("-----------------------------------------------------------")
        log.info("# Avaliando: Estimator={0} e Score = {1}".format(estimator["name"], score))

        clf = GridSearchCV(estimator["estimator"], estimator["parameters"], cv=2, verbose=5,  
                   scoring='%s' % score, n_jobs=-1)
        log.info("Criado o objeto GridSearchCV:{0}".format(clf))

        clf.fit(train_X, train_y)
        log.info("Concluído o 'fit' com os dados de treino.")

        log.info("Melhore estimator encontrado: {0}".format(clf.best_estimator_))
        log.info("Melhores parâmetros encontrados: {0}".format(clf.best_params_))
        log.info("Melhore score encontrado: {0:9.3f}".format(clf.best_score_))
        
        #Guarda os melhores parametros desta execução
        best_params.append("score={sc} - {bs:9.3f} -> {be}".format(sc = score, 
                                                                      bs=clf.best_score_, 
                                                                      be=clf.best_estimator_))
        # [clf.best_params_, clf.best_score_, clf.best_estimator_]
        
        
        log.info("Resultados dos testes com os diversos parâmetros: ")
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            log.info("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))

# exibe no log os melhores parâmetros de todas as execuções
log.info("")
log.info("-----------------------------------------------------------")
log.info("Melhores parâmetros de todas as execuções: ")
for be in best_params:
    log.info(be)
log.info("")

# maca fim do bloco e tempo gasto
log.info("Tempo gasto na Parte 3: %s" % (datetime.now() - t1))


####################################################################
# Fim do estudo
log.info("")
log.info("------------------------------------------------------")
log.info("")
log.info("Finish!")
